# Code Willing CDS Technical Interview

Below is historical raw security reference data. This data is a time series that contains every security that has traded on the New York Stock Exchange and the various identifiers that are associated with them. It also has a change date column, `Changed`, that lists the date of the changes to the identifiers over time. These identifiers are used to join different datasets or tables together when building a quantatative model. We will explore some of these identifiers below:

In [1]:
import numpy as np
import pandas as pd

pd.set_option('mode.chained_assignment', 'raise')
pd.options.display.max_columns = None
pd.options.mode.use_inf_as_na = True

In [2]:
sample = pd.read_csv('../data/pit_sample_xnys.txt', sep='\t', skiprows=1)
sample = sample[(sample.SecID.notna()) & (sample.IssID.notna())] # filter out rows with missing identifiers
sample.loc[:, ['SecID', 'IssID']] = sample.loc[:, ['SecID', 'IssID']].astype(int) # convert id columns to int
sample.head()

,ActFlag,Changed,SecID,IssID,Isin,Uscode,FISN,IssuerName,CntryofIncorp,CntryIncorpNumber,CntryofDom,StateofDom,LEI,FinancialYearEnd,ShellComp,SIC,CIK,GICS,NAICS,CIC,CFIcode,SectyCD,SecurityDesc,StructCD,ParValue,PVCurrency,StatusFlag,PrimaryExchgCD,Voting,VotePerSec,SharesOutstanding,EffectiveDate
0,U,2022/08/03 14:14:40,1399,1379,PHY078321437,NaN,NaN,Benguet Corporation,PH,NaN,NaN,NaN,NaN,3112.0,NaN,NaN,11290.0,15104020.0,212311.0,PH31,ESVUFR,EQS,Ordinary Shares - Class B,NaN,1.00,PHP,A,PHPSE,V,1.0,247926121.0,2021/10/08
1,U,2022/08/23 04:41:39,2234,2112,CA8667961053,866796105,NaN,"Sun Life Financial, Inc.",CA,NaN,NaN,NaN,549300V040JZX5S66Z78,3112.0,NaN,6311.0,1097362.0,40301020.0,524113.0,CA31,ESVTFR,EQS,Ordinary Shares,NaN,NaN,CAD,A,CATSE,V,1.0,586079916.0,2022/08/22
2,U,2009/12/14 22:54:13,4962,4788,GB0003188660,NaN,NaN,Enterprise Oil plc,GB,NaN,NaN,NaN,213800WIHSFSGMYPTK39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EQS,Ordinary Shares,NaN,0.25,GBP,I,GBLSE,V,NaN,NaN,NaN
3,U,2019/11/15 17:29:28,5246,5050,GB0004089834,NaN,NaN,Hanson plc,GB,NaN,NaN,NaN,529900N6RQR0PX7ZTR62,NaN,NaN,3822.0,NaN,NaN,334512.0,NaN,ESVUFR,EQS,Ordinary Shares,NaN,2.00,GBP,I,NaN,V,NaN,736968849.0,NaN
4,U,2010/08/20 18:43:01,5922,5692,GB0004437462,NaN,NaN,Powergen plc,GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EQS,Ordinary Shares,NaN,0.50,GBP,I,GBLSE,V,NaN,NaN,NaN


In [3]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10306 entries, 0 to 10305
Data columns (total 32 columns):
ActFlag              10306 non-null object
Changed              10306 non-null object
SecID                10306 non-null int64
IssID                10306 non-null int64
Isin                 9946 non-null object
Uscode               9881 non-null object
FISN                 13 non-null object
IssuerName           10306 non-null object
CntryofIncorp        10306 non-null object
CntryIncorpNumber    25 non-null object
CntryofDom           1845 non-null object
StateofDom           1665 non-null object
LEI                  5463 non-null object
FinancialYearEnd     8508 non-null float64
ShellComp            23 non-null object
SIC                  7466 non-null float64
CIK                  8464 non-null float64
GICS                 6021 non-null float64
NAICS                7021 non-null float64
CIC                  6151 non-null object
CFIcode              6545 non-null object
SectyC

Below we made a list of the important identifers in the dataset, as well as the change date. `SecID` is the security identifier, `IssID` is the company identifier, `Isin` is an international identifier, and `SIC, CIK, GICS, and NAICS` are all additional identifiers used to join datasets together.

In [4]:
ids = ['IssuerName', 'Changed', 'SecID', 'IssID', 'CntryofIncorp', 'Isin', 'SIC', 'CIK', 'GICS', 'NAICS']

In [5]:
example = (sample.query('IssID == 2042')
                 .loc[:, ids]
                 .sort_values(['IssID', 'Changed'])
                 .reset_index(drop=True))
example

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS
0,PLDT Inc,2006/01/11 21:16:24,89317,2042,PH,US7182527033,4813.0,78150.0,50102010.0,517210.0
1,PLDT Inc,2022/07/11 05:06:21,59927,2042,PH,US69344D4088,4813.0,78150.0,50102010.0,517210.0


In [6]:
example2 = (sample.query('IssID == 4121')
                 .loc[:, ids]
                 .sort_values(['IssID', 'Changed'])
                 .reset_index(drop=True))
example2

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS
0,Santander UK Plc,2003/12/02 10:05:46,92117,4121,GB,US0029202050,6029.0,1087711.0,40101010.0,522110.0
1,Santander UK Plc,2006/11/28 22:50:22,75715,4121,GB,US0029207000,6029.0,1087711.0,40101010.0,522110.0
2,Santander UK Plc,2007/10/08 13:00:37,83287,4121,GB,US0029208099,6029.0,1087711.0,40101010.0,522110.0


One of the main issues we address is when a company undergoes a corporate action (such as a split or merger), the company's underlying securities will often change identifiers. However, if you owned the security before the corporate action, you will still own it afterwards. In the above examples, the `SecID` changes, when it likely should have remained the same.

You can see for both companies, the `SecID` and `ISIN` changed on different dates, but the rest of the identifiers remained the same. This likely means that these sets of rows for `IssID == 2042` and `IssID == 4121` are actually the same security, and can be joined by our `CWIQ code`. Below I will provide a quick example of what this looks like:

In [7]:
_example = example.assign(cwiq_code=1001)
_example2 = example2.assign(cwiq_code=1002)
sec_master = pd.concat([_example, _example2])
sec_master.head()

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS,cwiq_code
0,PLDT Inc,2006/01/11 21:16:24,89317,2042,PH,US7182527033,4813.0,78150.0,50102010.0,517210.0,1001
1,PLDT Inc,2022/07/11 05:06:21,59927,2042,PH,US69344D4088,4813.0,78150.0,50102010.0,517210.0,1001
0,Santander UK Plc,2003/12/02 10:05:46,92117,4121,GB,US0029202050,6029.0,1087711.0,40101010.0,522110.0,1002
1,Santander UK Plc,2006/11/28 22:50:22,75715,4121,GB,US0029207000,6029.0,1087711.0,40101010.0,522110.0,1002
2,Santander UK Plc,2007/10/08 13:00:37,83287,4121,GB,US0029208099,6029.0,1087711.0,40101010.0,522110.0,1002


In [8]:
sec_master.query('cwiq_code == 1001')

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS,cwiq_code
0,PLDT Inc,2006/01/11 21:16:24,89317,2042,PH,US7182527033,4813.0,78150.0,50102010.0,517210.0,1001
1,PLDT Inc,2022/07/11 05:06:21,59927,2042,PH,US69344D4088,4813.0,78150.0,50102010.0,517210.0,1001


In [9]:
sec_master.query('cwiq_code == 1002')

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS,cwiq_code
0,Santander UK Plc,2003/12/02 10:05:46,92117,4121,GB,US0029202050,6029.0,1087711.0,40101010.0,522110.0,1002
1,Santander UK Plc,2006/11/28 22:50:22,75715,4121,GB,US0029207000,6029.0,1087711.0,40101010.0,522110.0,1002
2,Santander UK Plc,2007/10/08 13:00:37,83287,4121,GB,US0029208099,6029.0,1087711.0,40101010.0,522110.0,1002


As you can see, our cwiq_code allows us to query the data and pull all relevant information for a security, even if the vendor considers them two (or more) separate securities.

Your task is to provide:
- A data quality analysis on the data to determine the best way to identify which securities should be joined together. In the examples above, all the identifiers outside of `SecID` and `Isin` remained the same. But in other cases, other identifiers might changes as well, such as `GICS` or `SIC`. If `IssID` is the same, but the rest of the identifiers are different, then the two `SecID`s likely should get their own `cwiq_code` and not be joined together. Determine which columns are the best for determining if the securities are the same. In the cells below, show your analysis in determining the best way to identify these securities.
- Once you complete your analysis, develop a function that will
    1. Read in the data
    2. Extract the securities that need to be joined by `cwiq_code`
    3. Output a csv file of these securities to the folder called `output/`.

There is not one correct answer for this analysis. We are mostly concerned about seeing your programming abilities in python, and well as how you approach problems.

*You do not need to provide the `cwiq_code` itself, just the analysis, function, and output file of the securities that need to be joined*

The output below is an example of what your data should look like when you output it to csv

In [10]:
pd.concat([example, example2])

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS
0,PLDT Inc,2006/01/11 21:16:24,89317,2042,PH,US7182527033,4813.0,78150.0,50102010.0,517210.0
1,PLDT Inc,2022/07/11 05:06:21,59927,2042,PH,US69344D4088,4813.0,78150.0,50102010.0,517210.0
0,Santander UK Plc,2003/12/02 10:05:46,92117,4121,GB,US0029202050,6029.0,1087711.0,40101010.0,522110.0
1,Santander UK Plc,2006/11/28 22:50:22,75715,4121,GB,US0029207000,6029.0,1087711.0,40101010.0,522110.0
2,Santander UK Plc,2007/10/08 13:00:37,83287,4121,GB,US0029208099,6029.0,1087711.0,40101010.0,522110.0


To submit this code, please follow these steps:
    
1. Create a new git repository called "cw-technical-{first_name}_{last_name}" in either gitlab or github (your choice)
2. Ensure the repository you created is public
3. Add your code and altered data to this repository. The code should be in a folder called `src/` and the data should be in a folder called `output/`. Push these changes.
4. Please forward the link to your public git repository to dalton.hall@codewilling.com and cody.bordelon@codewilling.com

# Data Analysis

# ETL Function